# lc_gtis.ipynb
Abbie Stevens (<abigailstev@gmail.com>), 2022

In [1]:
import os
from astropy.table import Table ## to use astropy tables as our data storage and interaction format
import numpy as np

In [2]:
homedir = os.path.expanduser('~')
exe_dir = os.getcwd()
obj_name = "Swift_J0243.6+6124"
obj_prefix = "J0243"
data_dir = '%s/Reduced_data/%s/' % (homedir, obj_name)
assert os.path.isdir(data_dir)

These next steps are all for an individual obsid, since I am only using one observation in this paper.

In [3]:
obsid = "1050390113"

# Part 1
Making a light curve of each FPM detector to check for flares and dips due to instrument malfunction or space weather.

Setting up, assuming same internal file structure as in the FTP download from HEASARC

In [ ]:
obs_dir = '%s%s/xti/event_cl/' % (data_dir, obsid)
data_file = '%sni%s_0mpu7_cl.evt' % (obs_dir, obsid)
assert os.path.isfile(data_file), 'Data file does not exist.'
red_script = data_dir+"reduce.sh"
red_log = data_dir+"reduce.log"
local_tot_evt_file = "%s/xti/event_cl/obs%s_0mpu7_cl.evt" % (obsid, obsid[-3:])
local_ci_evt_file = "%s/xti/event_cl/obs%s_0mpu7_CI_cl.evt" % (obsid, obsid[-3:])

In [ ]:
detid_bin_file = exe_dir +"/in/detectors.txt"
## Could otherwise use n_chans = detchans FITS keyword in rsp matrix, and
## chan_bins=np.arange(detchans+1)  (need +1 for how histogram does ends)
detID_bins = np.loadtxt(detid_bin_file, dtype=int)

### Reading in event list and gti HDU

In [ ]:
datatab = Table.read(data_file, format='fits', hdu=1)
energy_mask = (datatab['PI'] >= 20) & (datatab['PI'] <= 1200)
datatab = datatab[energy_mask]

In [ ]:
gtitab = Table.read(data_file, format='fits', hdu=2)

In [ ]:
## Initializing
overall_means = np.asarray([])
overall_stds = np.asarray([])
det_means = np.asarray(np.zeros((len(detID_bins)-1,1)))  # -1 because of 0-indexed vs 1-indexed
det_stds = np.asarray(np.zeros((len(detID_bins)-1,1)))

In [ ]:
def det_lcs(times, dets, det_bins, gti_start, gti_stop, dt=1):
    """
    times: array of the event list times (arb start, same units as GTI times, in seconds)
    dets: array of the event list FPMs
    gti_start: the start of the GTI (arb start, in seconds)
    gti_stop: the end of the GTI (also in seconds)
    dt: the timestep of the desired lightcurve, in seconds
    """
    n_bins = int(np.round((gti_stop - gti_start) / dt))
    t_bin_seq = np.linspace(gti_start, gti_stop, num=n_bins + 1)
    
    lc_2d, t_bin_edges, d_bin_edges = np.histogram2d(times, dets,
                                                     bins=[t_bin_seq,
                                                           det_bins],
                                                     normed=False)
    ## Need counts/dt to have units of counts per second
    ## Doing it by multiplying by 1/dt, to keep it as an int and not get
    ## typecasting errors.
    dt_inv_int = np.int64(1. / dt)
    lc_2d *= dt_inv_int
    return lc_2d # in units of counts per second

### Looping through the GTIs for the data file

In [ ]:
for starttime, stoptime in zip(gtitab['START'], gtitab['STOP']):
    lc = det_lcs(datatab['TIME'],
                 datatab['DET_ID'], 
                 detID_bins, 
                 starttime, 
                 stoptime,
                 dt=0.5)
    print(np.shape(lc))
    overall_means = np.append(overall_means, np.mean(lc))
    overall_stds = np.append(overall_stds, np.std(lc))
    det_means = np.append(det_means, np.mean(lc, axis=0)[:, np.newaxis], axis=1)
    det_stds = np.append(det_stds, np.std(lc, axis=0)[:, np.newaxis], axis=1)
## Removing the zeros i initialized the 2-d arrays with
det_means = det_means[:,1:]
det_stds = det_stds[:,1:]

### Determining the upper and lower limits for 'flares' and 'dips' of grumpy FPMs

In [ ]:
upper_lim = overall_means + overall_stds
lower_lim = overall_means - overall_stds
# print(upper_lim)
# print(lower_lim)
above = np.where(det_means >= upper_lim)
below = np.where(det_means <= lower_lim)
# print(det_means[above])
flarey_bois = np.unique(detID_bins[above[0]])
print("FLAREY BOIS: ", flarey_bois)
# print(det_means[below])
dippy_bois = np.unique(detID_bins[below[0]])
print("DIPPY BOIS: ", dippy_bois)

### Checking if all the problems are with the same GTIs. If so, throw it out!

In [ ]:
## Checking if all the problems are with the same GTIs. If so, throw it out!
print("But wait, is it a GTI issue?")
## Remember that these are zero-indexed, so [3] means the 4th GTI in the list.
print("GTIs with flarey bois: ", np.unique(above[1]))
print("GTIs with dippy bois: ", np.unique(below[1]))

In [ ]:
del datatab
del gtitab

## Running nicerl2
If you identified bad FPMs, remove them here with expr='(DET_ID != XX)'

(nicercal, niextract-events, nimaketime, and nicer-mergeclean with min_fpm=7) to get cleaned event lists

In [5]:
with open(red_script, 'w') as f:
    f.write("if [ -e %s ]; \n" % red_log)
    f.write("then rm %s\n" % red_log)
    f.write("fi \n")
#     f.write("nicerl2 %s clobber=yes > %s\n" % (str(obsid), redlog)) 
#     f.write("fselect infile=./%s/xti/event_cl/ni%s_0mpu7_cl.evt "
#             "outfile=./%s "
#             "clobber=yes > %s\n" % (str(obsid), str(obsid), local_tot_evt_file, red_log))
    f.write("cp ./%s/xti/event_cl/ni%s_0mpu7_cl.evt ./%s \n" % (obsid, obsid, local_tot_evt_file)) 
    f.write("fselect infile=./%s/xti/event_cl/ni%s_0mpu7_cl.evt "
            "outfile=./%s "
            "expr='(DET_ID < 40)' "
            "clobber=yes > %s\n" % (obsid, obsid, local_ci_evt_file, red_log))
print("Run these things at the command line:")
print("bash")
print("heainit")
print("cd %s" % data_dir)
print("chmod u+x %s" % os.path.basename(red_script))
print("./%s" % os.path.basename(red_script))

Run these things at the command line:
bash
heainit
cd /home/astevens/Reduced_data/Swift_J0243.6+6124/
chmod u+x reduce.sh
./reduce.sh


# Part 2
Getting the exact GTIs we will use for the timing analysis, so that we're making spectra with only those exact photons. 

In [ ]:
from stingray.events import EventList
from stingray.lightcurve import Lightcurve

In [ ]:
data_file = '%s%s' % (data_dir, local_tot_evt_file)
assert os.path.isfile(data_file), 'Data file does not exist.'
local_gti_file = "%s/xti/event_cl/obs%s_nobary.gti" % (obsid, obsid[-3:])
gti_file = "%s%s" % (data_dir, local_gti_file)
extr_script = data_dir+"extract.sh"
extr_log = data_dir+"extract.log"
local_extr_file = "%s/xti/event_cl/obs%s_nobary-extr.evt" % (obsid, obsid[-3:])
local_ci_extr_file = "%s/xti/event_cl/obs%s_CI_nobary-extr.evt" % (obsid, obsid[-3:])

In [ ]:
datatab = Table.read(data_file, format='fits', hdu=1)
print(len(datatab))
energy_mask = (datatab['PI'] >= 20) & (datatab['PI'] <= 1200)
datatab = datatab[energy_mask]
print(len(datatab))
## number should go down, because we're removing events

In [ ]:
gtitab = Table.read(data_file, format='fits', hdu=2)
print(gtitab)
gtis = [[i,j] for i,j in zip(gtitab['START'], gtitab['STOP'])]

In [ ]:
lc = Lightcurve.make_lightcurve(datatab['TIME'], dt=1./2, gti=gtis)

In [ ]:
lc.plot()

In [ ]:
def meanseg(lc):
    return np.mean(lc.countrate)

In [ ]:
n_sec = 64.0
start, stop, meanrate = lc.analyze_lc_chunks(segment_size=n_sec, func=meanseg)
print(len(start))

In [ ]:
print(meanrate)

In [ ]:
print(stop-start)
print(start[1]-start[0])

In [ ]:
ends = stop-lc.dt
print(np.mean(ends-start))

If you want to save your segment starts and ends to a text file, uncomment the next block.

In [ ]:
# seg_times = np.column_stack((start, ends))
# gti_txt = exe_dir+"/out/%s_%dsec_%ddt_seg-times.txt" % (obj_prefix, n_sec, int(1/lc.dt))
# np.savetxt(gti_txt, seg_times, fmt="%.12f")

Run this at the command line to get a hollow husk of a GTI file 

In [6]:
print("Run these things at the command line:")
print("bash")
print("heainit")
print("cd %s" % data_dir)
print("nimaketime infile=./%s/auxil/ni%s.mkf outfile=./%s" % (obsid, obsid, local_gti_file))

Run these things at the command line:
bash
heainit
cd /home/astevens/Reduced_data/Swift_J0243.6+6124/
nimaketime infile=./1050390113/auxil/ni1050390113.mkf outfile=./1050390113/xti/event_cl/obs113_nobary.gti


In [ ]:
assert os.path.isfile(gti_file), 'GTI file does not exist.'

In [ ]:
gti2 = Table.read(gti_file, format='fits')
print(gti2)

In [ ]:
final_tab = Table()
final_tab.meta = gti2.meta

In [ ]:
final_tab['START'] = start
final_tab['STOP'] = ends

In [ ]:
final_tab.write(gti_file, format='fits', overwrite=True)

In [ ]:
del datatab
del gtitab

## Extract the events within just those specific GTIs

In [7]:
with open(extr_script, 'w') as f:
    f.write("if [ -e %s ]; \n" % extr_log)
    f.write("then rm %s\n" % extr_log)
    f.write("fi \n")
    f.write("niextract-events "
            "filename=./%s[PI=20:1200,EVENT_FLAGS=bxxx1x000] "
            "eventsout=./%s "
            "timefile=./%s "
            "gti=GTI clobber=YES chatter=5 > %s\n" % (local_tot_evt_file, local_extr_file, local_gti_file, extr_log))
    f.write("niextract-events "
            "filename=./%s[PI=20:1200,EVENT_FLAGS=bxxx1x000] "
            "eventsout=./%s "
            "timefile=./%s "
            "gti=GTI clobber=YES chatter=5 > %s" % (local_ci_evt_file, local_ci_extr_file, local_gti_file, extr_log))
print("Run these things at the command line:")
print("bash")
print("heainit")
print("cd %s" % data_dir)
print("chmod u+x %s" % os.path.basename(extr_script))
print("./%s" % os.path.basename(extr_script))

Run these things at the command line:
bash
heainit
cd /home/astevens/Reduced_data/Swift_J0243.6+6124/
chmod u+x extract.sh
./extract.sh


## Run xselect in a terminal window (haven't scripted this)

In [14]:
spec_file = "%s/obs113_tot.pha" % data_dir
ci_spec_file = "%s/obs113_CI_tot.pha" % data_dir

In [8]:
assert os.path.isfile(spec_file)
assert os.path.isfile(ci_spec_file)

# Part 3
Running NICER background estimator, and rebinning total and background spectra to chbinfile-6.txt.

Following instructions in the README file that comes with the 3C50 tarball. Altered the code so I could specify my own output file.

In [9]:
from nicergof.bkg import bkg_estimator as be

In [24]:
bevt_file = "%s/Documents/Research/nicergof/30nov18targskc_enhanced.evt" % homedir
assert os.path.isfile(bevt_file)
mkf_file = "%s%s/auxil/ni%s.mkf" % (data_dir, obsid, obsid)
bkgd_file = "%sobs113_bkgd.pha" % data_dir
ci_bkgd_file = "%sobs113_CI_bkgd.pha" % data_dir
rbn_script = data_dir+"rbn_pha.sh"
rbn_spec_file = "%sobs113_tot_rbn-6.pha" % data_dir
rbn_ci_spec_file = "%sobs113_CI_tot_rbn-6.pha" % data_dir
rbn_bkgd_file = "%sobs113_bkgd_rbn-6.pha" % data_dir
rbn_ci_bkgd_file = "%sobs113_CI_bkgd_rbn-6.pha" % data_dir

In [21]:
status = be.add_kp(mkf_file)

Writing /home/astevens/Reduced_data/Swift_J0243.6+6124/1050390113/auxil/ni1050390113.mkf3
File /home/astevens/Reduced_data/Swift_J0243.6+6124/1050390113/auxil/ni1050390113.mkf3 already exists. If you mean to replace it then use the argument "overwrite=True".


In [22]:
bkg_chan, bkgspectot, btotexpo = be.mk_bkg_spec_evt(spec_file, mkf3file="%s3" % mkf_file, 
                                                    bevt=bevt_file, outfile=bkgd_file, verbose=False)

For GTI #0; Duration = 512.0 start:120970496.00001875 stop:120971008.00001875
For GTI #1; Duration = 320.0 start:120976287.00001875 stop:120976607.00001875
For GTI #2; Duration = 832.0 start:120992532.00001875 stop:120993364.00001875
For GTI #3; Duration = 128.0 start:120993418.50001875 stop:120993546.50001875
For GTI #4; Duration = 1024.0 start:121009193.00001875 stop:121010217.00001875
For GTI #5; Duration = 896.0 start:121014747.00001875 stop:121015643.00001875
Binning spectrum from PI column
Done


In [23]:
bkg_chan, bkgspectot, btotexpo = be.mk_bkg_spec_evt(ci_spec_file, mkf3file="%s3" % mkf_file, 
                                                    bevt=bevt_file, outfile=ci_bkgd_file, verbose=False)

For GTI #0; Duration = 512.0 start:120970496.00001875 stop:120971008.00001875
For GTI #1; Duration = 320.0 start:120976287.00001875 stop:120976607.00001875
For GTI #2; Duration = 832.0 start:120992532.00001875 stop:120993364.00001875
For GTI #3; Duration = 128.0 start:120993418.50001875 stop:120993546.50001875
For GTI #4; Duration = 1024.0 start:121009193.00001875 stop:121010217.00001875
For GTI #5; Duration = 896.0 start:121014747.00001875 stop:121015643.00001875
Binning spectrum from PI column
Done


In [27]:
with open(rbn_script, 'w') as f:
    f.write("rbnpha %s %s binfile=chbinfile-6.txt error=poiss-0 clobber=yes \n" % (spec_file, rbn_spec_file))
    f.write("rbnpha %s %s binfile=chbinfile-6.txt error=poiss-0 clobber=yes \n" % (ci_spec_file, rbn_ci_spec_file))
    f.write("rbnpha %s %s binfile=chbinfile-6.txt error=poiss-0 clobber=yes \n" % (bkgd_file, rbn_bkgd_file))
    f.write("rbnpha %s %s binfile=chbinfile-6.txt error=poiss-0 clobber=yes \n" % (ci_bkgd_file, rbn_ci_bkgd_file))

print("Run these things at the command line:")
print("bash")
print("heainit")
print("cd %s" % data_dir)
print("chmod u+x %s" % os.path.basename(rbn_script))
print("./%s" % os.path.basename(rbn_script))

Run these things at the command line:
bash
heainit
cd /home/astevens/Reduced_data/Swift_J0243.6+6124/
chmod u+x rbn_pha.sh
./rbn_pha.sh
